# Analyze SARS-CoV-2 Spike Glycoprotein Variants
[Work in progress]

This notebook queries the Knowledge Graph for variants in the SARS-CoV S gene and analyzes mutations that may affect the ACE2 binding to the Spike Glycoprotein as well we mutations at the polybasic cleavage site. In addition, it analyzes the geographic locations of the variants.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from py2neo import Graph

In [27]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [28]:
import py2neo
py2neo.__version__

'2020.7b6'

#### Connect to COVID-19-Community Knowledge Graph

In [29]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

## Analysis of missense mutations in the SARS-CoV-2 Spike glycoprotein

![Spike protein](../../docs/Spikeprotein.png)

**a.** ACE2 receptor binding domain of the SARS-CoV-2 spike protein. **b.** Polybasic cleavage site with three predicted O-glycosylation sites.

Reference: Andersen, K.G., Rambaut, A., Lipkin, W.I. et al. The proximal origin of SARS-CoV-2. Nat Med 26, 450–452 (2020). [doi:10.1038/s41591-020-0820-9](https://doi.org/10.1038/s41591-020-0820-9)

### Query KG for the S gene and its gene product

Mutation data below are based on the Wuhan-Hu-1 ([NC_045512](http://identifiers.org/resolve?query=ncbiprotein:NC_045512)) reference sequence.

In [30]:
geneName = 'S'
genomeAccession = 'ncbiprotein:NC_045512'

In [31]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:ENCODES]->(p:Protein) 
RETURN g.id as geneAccession, g.name AS geneName, p.accession AS proteinAccession, p.sequence AS proteinSequence
"""
df = graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()

In [32]:
df.head()

,geneAccession,geneName,proteinAccession,proteinSequence
0,ncbiprotein:NC_045512-21563-25384,S,uniprot:P0DTC2,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [33]:
seq = df['proteinSequence'].iloc[0]

In [34]:
# TODO check why this sequence is shorter than the one in the figure above.
print('Sequence of the Spike glycoprotein:', len(seq), 'residues')
print(seq)

Sequence of the Spike glycoprotein: 1273 residues
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQD

### Type of Variants in Spike glycoprotein

List of variant types and consequences:

https://uswest.ensembl.org/info/genome/variation/prediction/classification.html

https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html#consequences

In [35]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant)
       <-[:HAS_VARIANT]-(s:Strain)
WHERE s.hostTaxonomyId = 'taxonomy:9606'
RETURN DISTINCT(v.variantType), v.variantConsequence, count(s)
ORDER BY v.variantType
"""

In [36]:
df = graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()

In [37]:
df.head(20)

,(v.variantType),v.variantConsequence,count(s)
0,Deletion,frameshift_variant,12
1,Deletion,inframe_deletion,81
2,Insertion,frameshift_variant,7
3,Insertion,inframe_insertion,1
4,SNP,synonymous_variant,4899
5,SNP,missense_variant,29078
6,SNP,coding_sequence_variant,1312
7,SNP,stop_gained,15
8,SNP,stop_retained_variant,1
9,SNP,stop_lost,13


### Frequency of mutation by sequence position
**NOTE, some strains have been deposited in both in GISAID and NCBI under different strain ids. They will be double-counted in the analyses below.**

Receptor-binding domain contact residues and O-linked glycan sites are highlighted in the plot below.

In [38]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})
       <-[:HAS_VARIANT]-(s:Strain)
WHERE s.hostTaxonomyId = 'taxonomy:9606'
RETURN v.proteinPosition AS proteinPosition, v.geneVariant as geneVariant, v.proteinVariant AS proteinVariant, 
       s.name as strain, s.collectionDate AS collectionDate
ORDER by geneVariant
"""

In [39]:
df = graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()

In [40]:
positions = df['proteinPosition'].values

In [1]:
x = np.arange(1285)
y = np.zeros(1285)
for p in positions:
    y[p] += 1

plt.rcParams['figure.dpi'] = 300
plt.rcParams["figure.figsize"] = (20,8)

plt.plot(x, y)
plt.yscale('log')
plt.xlabel('Sequence position')
plt.ylabel('Number of mutations')
plt.title('Mutations by Sequence Position')

# receptor-binding domain (RBD) contact residues in red
rbd = (x >= 455) & (x <= 505)
plt.scatter(x[rbd], y[rbd], color='red') 

# O-linked glycan sites in orange
ogs = (x >= 673) & (x <= 686)
plt.scatter(x[ogs], y[ogs], color='darkorange') 

plt.annotate('614', xy=(600, 17000))
plt.annotate('Receptor-binding domain', xy=(400, 100), color='red')
plt.annotate('O-linked glycan sites', xy=(620, 100), color='darkorange')

plt.show()

NameError: name 'np' is not defined

##### Mutations at position 614

In [42]:
df.query('proteinPosition == 614').head()

,proteinPosition,geneVariant,proteinVariant,strain,collectionDate
1151,614,S:c.1840Gat>Aat,QHD43416.1:p.614D>N,hCoV-19/Wales/PHWC-160270/2020,2020-04-19
1152,614,S:c.1840Gat>Aat,QHD43416.1:p.614D>N,hCoV-19/England/CAMB-81F82/2020,2020-04-21
1153,614,S:c.1840Gat>Aat,QHD43416.1:p.614D>N,hCoV-19/England/CAMB-81E94/2020,2020-04-22
1154,614,S:c.1841gAt>gGt,QHD43416.1:p.614D>G,SARS-CoV-2/human/BGD/BCSIR_NILMRC_257/2020,2020-07-07
1155,614,S:c.1841gAt>gGt,QHD43416.1:p.614D>G,hCoV-19/USA/WA-UW-3158/2020,2020-03-28


### Strains with mutations in the Spike glycoprotein receptor-binding domain (RBD)

Six amino acids in the RBD of the spike protein have been shown to be critical for binding to ACE2 receptors: 

L455, F486, Q493, S494, N501, Y505

Python uses zero-based indices, so we subtract 1 to find the position in the sequence.

In [43]:
print(seq[455-1], seq[486-1], seq[493-1], seq[494-1], seq[501-1], seq[505-1])

L F Q S N Y


#### The following variants are missense mutations in the RBD

In [44]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})
<-[:HAS_VARIANT]-(s:Strain)-[:FOUND_IN]->(l:Location)
WHERE v.proteinPosition IN [455, 486, 493, 494, 501, 505] AND s.hostTaxonomyId = 'taxonomy:9606'
RETURN DISTINCT v.geneVariant as geneVariant, v.proteinVariant AS proteinVariant, 
s.name AS strainName, s.collectionDate AS collectionDate, l.name AS location, 
labels(l) AS locationType, s.id AS strainId
"""
graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()

,geneVariant,proteinVariant,strainName,collectionDate,location,locationType,strainId
0,S:c.1481tCa>tTa,QHD43416.1:p.494S>L,hCoV-19/England/BRIS-12B47B/2020,2020-05-26,England,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_487571
1,S:c.1365ttG>ttT,QHD43416.1:p.455L>F,hCoV-19/Scotland/EDB2785/2020,2020-03-30,Scotland,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_489003
2,S:c.1365ttG>ttT,QHD43416.1:p.455L>F,hCoV-19/England/20168001704/2020,2020-04-16,England,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_466592
3,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2066/2020,2020-06-12,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480756
4,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2089/2020,2020-06-12,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480701
5,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2067/2020,2020-06-12,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480757
6,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2160/2020,2020-06-16,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480760
7,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2065/2020,2020-06-15,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480755
8,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2151/2020,2020-06-18,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480765
9,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2112/2020,2020-06-17,Victoria,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480718


#### Find variants around these critical residues within a 5 amino acid window (+/- 2 residues)

In [45]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})
<-[:HAS_VARIANT]-(s:Strain)-[:FOUND_IN]->(l:Location)
WHERE v.proteinPosition IN [453,454,455,456,457, 484,485,486,487,488, 491,492,493,494,495,496,
                            499,500,501,502,503,504,505,506,507] 
      AND s.hostTaxonomyId = 'taxonomy:9606'
RETURN DISTINCT v.geneVariant as geneVariant, v.proteinVariant AS proteinVariant, 
s.name AS strainName, s.collectionDate AS collectionDate, s.sex AS sex, s.age as age, 
l.name AS location, l.location AS coordinates, labels(l) AS locationType, s.id AS strainId
"""
df = graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()
df.fillna('', inplace=True)
df

,geneVariant,proteinVariant,strainName,collectionDate,sex,age,location,coordinates,locationType,strainId
0,S:c.1370aGg>aAg,QHD43416.1:p.457R>K,SARS-CoV-2/human/USA/FL-BPHL-0344/2020,2020-04-13,,,Florida,"(-82.5001, 28.75054)","[Location, Admin1]",insdc:MT757013
1,S:c.1370aGg>aAg,QHD43416.1:p.457R>K,SARS-CoV-2/human/USA/FL-BPHL-0344/2020,2020-04-13,,,Florida,"(-82.5001, 28.75054)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_489753
2,S:c.1481tCa>tTa,QHD43416.1:p.494S>L,hCoV-19/England/BRIS-12B47B/2020,2020-05-26,,,England,"(-0.70312, 52.16045)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_487571
3,S:c.1450Gaa>Caa,QHD43416.1:p.484E>Q,hCoV-19/Wales/PHWC-15FCEF/2020,2020-05-21,,,Wales,"(-3.5, 52.5)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_472846
4,S:c.1365ttG>ttT,QHD43416.1:p.455L>F,hCoV-19/Scotland/EDB2785/2020,2020-03-30,,,Scotland,"(-4.0, 56.0)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_489003
5,S:c.1365ttG>ttT,QHD43416.1:p.455L>F,hCoV-19/England/20168001704/2020,2020-04-16,,,England,"(-0.70312, 52.16045)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_466592
6,S:c.1450Gaa>Aaa,QHD43416.1:p.484E>K,hCoV-19/Sweden/20-51636/2020,2020-05-22,male,30,Stockholm,"(18.0, 59.5)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_476136
7,S:c.1450Gaa>Aaa,QHD43416.1:p.484E>K,hCoV-19/England/NOTT-111DB9/2020,2020-06-08,,,England,"(-0.70312, 52.16045)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_472405
8,S:c.1450Gaa>Aaa,QHD43416.1:p.484E>K,hCoV-19/England/NOTT-1115C0/2020,2020-05-18,,,England,"(-0.70312, 52.16045)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_461895
9,S:c.1501Aat>Tat,QHD43416.1:p.501N>Y,hCoV-19/Australia/VIC2066/2020,2020-06-12,,,Victoria,"(145.0, -37.0)","[Location, Admin1]",https://www.gisaid.org/EPI_ISL_480756


### Find strains with mutations in the polybasic cleavage site
This site has three predicted O-glycosylation sites:
    
S673, T678, S686

In [46]:
print(seq[673-1], seq[678-1], seq[686-1])

S T S


In [47]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})
<-[:HAS_VARIANT]-(s:Strain)-[:FOUND_IN]->(l:Location)
WHERE v.proteinPosition IN [673, 678, 686] 
      AND s.hostTaxonomyId = 'taxonomy:9606'
RETURN DISTINCT v.geneVariant as geneVariant, v.proteinVariant AS proteinVariant, 
s.name AS strainName, s.collectionDate AS collectionDate, s.sex AS sex, s.age as age, 
l.name AS location, labels(l) AS locationType, s.id AS strainId
"""
df = graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()
df.fillna('', inplace=True)
df

,geneVariant,proteinVariant,strainName,collectionDate,sex,age,location,locationType,strainId
0,S:c.2033aCt>aTt,QHD43416.1:p.678T>I,hCoV-19/Scotland/CVR2305/2020,2020-04-13,,,Scotland,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_477897
1,S:c.2033aCt>aTt,QHD43416.1:p.678T>I,hCoV-19/USA/CA-SR0181/2020,2020-03-24,,,San Diego,"[Location, City]",https://www.gisaid.org/EPI_ISL_437596


#### Find variants around these residues within a 5 amino acid window (+/- 2 residues)

In [48]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})
<-[:HAS_VARIANT]-(s:Strain)-[:FOUND_IN]->(l:Location)
WHERE v.proteinPosition IN [671,672,673,674,675, 676,677,678,679,670, 684,685,686,687,688] 
      AND s.hostTaxonomyId = 'taxonomy:9606'
RETURN DISTINCT v.geneVariant as geneVariant, v.proteinVariant AS proteinVariant, 
s.name AS strainName, s.collectionDate AS collectionDate, s.sex AS sex, s.age as age, l.name AS location, 
labels(l) AS locationType, s.id AS strainId
ORDER BY v.proteinVariant, s.collectionDate
"""
df = graph.run(query, geneName=geneName, genomeAccession=genomeAccession).to_data_frame()
df.fillna('', inplace=True)
df.head()

,geneVariant,proteinVariant,strainName,collectionDate,sex,age,location,locationType,strainId
0,S:c.2008Ata>Tta,QHD43416.1:p.670I>L,hCoV-19/England/NORW-E8DF8/2020,2020-04-26,,,England,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_453558
1,S:c.2008Ata>Tta,QHD43416.1:p.670I>L,hCoV-19/England/NORW-EA52B/2020,2020-06-17,,,England,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_478719
2,S:c.2010atA>atG,QHD43416.1:p.670I>M,hCoV-19/Scotland/CVR1898/2020,2020-04-08,,,Scotland,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_477828
3,S:c.2010atA>atG,QHD43416.1:p.670I>M,hCoV-19/Scotland/CVR2973/2020,2020-04-22,,,Scotland,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_448213
4,S:c.2010atA>atG,QHD43416.1:p.670I>M,hCoV-19/Scotland/CVR3170/2020,2020-04-27,,,Scotland,"[Location, Admin1]",https://www.gisaid.org/EPI_ISL_449200


### Mutations in the SARS-CoV-2 "S" gene at different geolocations

In [49]:
query = """
MATCH (g:Gene{name: $geneName, genomeAccession: $genomeAccession})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})
<-[:HAS_VARIANT]-(s:Strain)-[:FOUND_IN]->(l1:Location)-[:IN*]->(r:USRegion{name:$region})
MATCH (l1)-[:IN]->(l2)
RETURN v.name AS geneVariant, l1.name, l1.location, labels(l1), l2.name, labels(l2), r.name AS region
ORDER BY v.name, l1.name
"""

#### Variants in the US West Region

In [50]:
region = 'West Region'
west = graph.run(query, geneName=geneName, genomeAccession=genomeAccession, region=region).to_data_frame()
west.head()

,geneVariant,l1.name,l1.location,labels(l1),l2.name,labels(l2),region
0,S:c.100Cgt>Tgt,San Diego,"(-117.16472, 32.71571)","[Location, City]",San Diego County,"[Location, Admin2]",West Region
1,S:c.1025tTt>tGt,Los Angeles,"(-118.24368, 34.05223)","[Location, City]",Los Angeles County,"[Location, Admin2]",West Region
2,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",United States,"[Location, Country]",West Region
3,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",Pacific Division,"[Location, USDivision]",West Region
4,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",United States,"[Location, Country]",West Region


#### Variants in the US Northeast Region

In [51]:
region = 'Northeast Region'
northeast = graph.run(query, geneName=geneName, genomeAccession=genomeAccession, region=region).to_data_frame()
northeast.head()

,geneVariant,l1.name,l1.location,labels(l1),l2.name,labels(l2),region
0,S:c.1076aGc>aAc,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
1,S:c.1150Cct>Tct,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
2,S:c.1150Cct>Tct,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
3,S:c.13Ctt>Ttt,Brooklyn,"(-73.94958, 40.6501)","[Location, City]",Kings County,"[Location, Admin2]",Northeast Region
4,S:c.13Ctt>Ttt,Brooklyn,"(-73.94958, 40.6501)","[Location, City]",Kings County,"[Location, Admin2]",Northeast Region


#### Find variants in common

In [52]:
in_common = pd.merge(west, northeast, on='geneVariant')
in_common[['geneVariant']].drop_duplicates().head(25)

,geneVariant
0,S:c.13Ctt>Ttt
946,S:c.1558Gca>Tca
956,S:c.1709gCt>gTt
960,S:c.1841gAt>gGt
5488710,S:c.2117gCt>gTt
5488711,S:c.2576aCt>aTt
5488756,S:c.3058Gct>Tct
5488758,S:c.3301Cac>Tac
5488785,S:c.3352Gac>Tac
5488797,S:c.3485cCa>cTa


#### Find unique variants in the West region

In [53]:
in_west_only = pd.merge(west, northeast, on='geneVariant', how='left')
in_west_only.head()

,geneVariant,l1.name_x,l1.location_x,labels(l1)_x,l2.name_x,labels(l2)_x,region_x,l1.name_y,l1.location_y,labels(l1)_y,l2.name_y,labels(l2)_y,region_y
0,S:c.100Cgt>Tgt,San Diego,"(-117.16472, 32.71571)","[Location, City]",San Diego County,"[Location, Admin2]",West Region,NaN,NaN,NaN,NaN,NaN,NaN
1,S:c.1025tTt>tGt,Los Angeles,"(-118.24368, 34.05223)","[Location, City]",Los Angeles County,"[Location, Admin2]",West Region,NaN,NaN,NaN,NaN,NaN,NaN
2,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",United States,"[Location, Country]",West Region,NaN,NaN,NaN,NaN,NaN,NaN
3,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",Pacific Division,"[Location, USDivision]",West Region,NaN,NaN,NaN,NaN,NaN,NaN
4,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",United States,"[Location, Country]",West Region,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
in_west_only.fillna('', inplace=True)
in_west_only.query("`l1.name_y` == ''", inplace=True)
in_west_only = in_west_only[['geneVariant', 'l1.name_x', 'l1.location_x', 'labels(l1)_x', 'l2.name_x', 'labels(l2)_x', 'region_x']]
in_west_only.head(25)

,geneVariant,l1.name_x,l1.location_x,labels(l1)_x,l2.name_x,labels(l2)_x,region_x
0,S:c.100Cgt>Tgt,San Diego,"(-117.16472, 32.71571)","[Location, City]",San Diego County,"[Location, Admin2]",West Region
1,S:c.1025tTt>tGt,Los Angeles,"(-118.24368, 34.05223)","[Location, City]",Los Angeles County,"[Location, Admin2]",West Region
2,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",United States,"[Location, Country]",West Region
3,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",Pacific Division,"[Location, USDivision]",West Region
4,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",United States,"[Location, Country]",West Region
5,S:c.1042Gca>Aca,Washington,"(-120.50147, 47.50012)","[Location, Admin1]",Pacific Division,"[Location, USDivision]",West Region
6,S:c.1099Gtc>Ttc,Los Angeles,"(-118.24368, 34.05223)","[Location, City]",Los Angeles County,"[Location, Admin2]",West Region
7,S:c.1127aCt>aTt,Los Angeles,"(-118.24368, 34.05223)","[Location, City]",Los Angeles County,"[Location, Admin2]",West Region
8,S:c.1151cCt>cTt,San Diego,"(-117.16472, 32.71571)","[Location, City]",San Diego County,"[Location, Admin2]",West Region
9,S:c.1169cTc>cCc,Los Angeles,"(-118.24368, 34.05223)","[Location, City]",Los Angeles County,"[Location, Admin2]",West Region


#### Find unique variants in the Northeast region

In [55]:
in_northeast_only = pd.merge(northeast, west, on='geneVariant', how='left')
in_northeast_only.fillna('', inplace=True)
in_northeast_only.query("`l1.name_y` == ''", inplace=True)
in_northeast_only = in_northeast_only[['geneVariant', 'l1.name_x', 'l1.location_x', 'labels(l1)_x', 'l2.name_x', 'labels(l2)_x', 'region_x']]
in_northeast_only.head(25)

,geneVariant,l1.name_x,l1.location_x,labels(l1)_x,l2.name_x,labels(l2)_x,region_x
0,S:c.1076aGc>aAc,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
1,S:c.1150Cct>Tct,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
2,S:c.1150Cct>Tct,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
949,S:c.1501Aat>Tat,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
950,S:c.1507Gtt>Ttt,Brooklyn,"(-73.94958, 40.6501)","[Location, City]",Kings County,"[Location, Admin2]",Northeast Region
961,S:c.1586aAg>aGg,Pittsburgh,"(-79.99589, 40.44062)","[Location, City]",Allegheny County,"[Location, Admin2]",Northeast Region
962,S:c.162ttG>ttT,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
963,S:c.1662gaG>gaT,Manhattan,"(-73.96625, 40.78343)","[Location, City]",New York County,"[Location, Admin2]",Northeast Region
964,S:c.1708Gct>Act,Queens,"(-73.83652, 40.68149)","[Location, City]",Queens County,"[Location, Admin2]",Northeast Region
969,S:c.1749gaG>gaT,Manhattan,"(-73.96625, 40.78343)","[Location, City]",New York County,"[Location, Admin2]",Northeast Region
